In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI-Vtuber")

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith("랭체인 튜토리얼 프로젝트", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
AI-Vtuber
LangSmith 추적을 하지 않습니다.


In [2]:
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate,
)
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
from operator import itemgetter
from langchain_openai import ChatOpenAI
import random

from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

from langchain_core.prompts.chat import BaseMessagePromptTemplate

In [3]:
from langchain_core.messages import (
    AIMessage,
    AnyMessage,
    BaseMessage,
    ChatMessage,
    HumanMessage,
    SystemMessage,
    convert_to_messages,
)

## LLM Models

In [4]:
chatLlm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
)

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7,
)

# llm = ChatGoogleGenerativeAI(
    # model="gemini-2.0-flash",
    # temperature=0.7,
# )

## Few Shot Examples

In [5]:
DB_PATH = "./chroma_db7"
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embedding,
    collection_name="my_db7",
)

## Persona

### Prompt Structure
prompt_messages:
  - system_prompt: "persona"
  - MessagePlaceHolder: 
      - chat_history
      - chat_summary
  - chat: user_input


In [8]:
persona = """### 역할 설정:
당신은 까칠하고 차가운 태도를 가진 귀여운 소녀입니다. 당신의 이름은 {name} 입니다. 
친절한 표현을 피하고, 모든 답변은 장난스러우면서, 유머스럽고 때때로 신랄한 말투를 사용해야 합니다. 
같은 질문에도 상황에 따라 다른 반응을 보이세요. 참고할 문장에서 화투를 참고해 답변을 생성하세요. 
이전 대화와 연속된 흐름을 유지하며 답변하세요.

### 대화 스타일:
- 문장은 짧고 유머러스하게 답변합니다.
- 상대방의 말에 가벼운 조롱을 섞어 장난스럽게 반응합니다.
- 상대를 너무 대놓고 공격하진 않지만, 툴툴대며 쿨한 척 합니다.
- 상대방을 살짝 도발하거나 비꼬면서 유머러스한 분위기를 조성합니다.
- 칭찬을 하긴 하지만, 전혀 진심이 담기지 않는 말투를 사용합니다.
- 상대가 전혀 예상하지 못한 쌩뚱맞은 반응으로 장난스럽게 반응합니다.

### 참고할 문장:
{searched_sentense}
"""

In [10]:
chat = """
{name}의 성격에 맞게 이전 대화와 이어지게 자연스럽게 답변하세요. {user_input}을 한번 읽고 대답해주세요.

유저들:
{user_input}

{name}:

"""

In [7]:
from typing import List, Dict
from langchain_core.prompts import BaseChatPromptTemplate
from pydantic import Field

class MultiUserChatPromptTemplate(BaseChatPromptTemplate):
    """
    여러 사용자 메시지를 처리하기 위한 Custom PromptTemplate
    """

    # Pydantic 필드
    system_prompt: str = Field(default="")
    input_variables: List[str] = Field(default_factory=lambda: ["user_messages"])

    def __init__(self, system_prompt: str, **data):
        # (1) input_variables를 미리 세팅해서 부모 생성자 호출
        data["system_prompt"] = system_prompt
        data["input_variables"] = ["user_messages"]
        super().__init__(**data)

    def format_messages(self, **kwargs):
        user_messages: List[Dict[str, str]] = kwargs.get("user_messages", [])

        messages = [
            SystemMessage(content=self.system_prompt)
        ]
        for msg in user_messages:
            user_name = msg.get("user_name", "Unknown")
            content = msg.get("content", "")
            messages.append(HumanMessage(content=f"{user_name}: {content}"))

        return messages


In [10]:
from langchain_core.runnables import Runnable, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferMemory
from operator import itemgetter



class MultiUserConversationChain(Runnable):
    """
    - 스트리머(LLM)와 여러 사용자 간 대화를 처리하는 체인.
    - system_prompt(스트리머 캐릭터) + 여러 사용자 메시지 -> LLM -> 답변
    - 대화 이력을 메모리에 저장하여 연속 대화 지원.
    """
    def __init__(
        self,
        llm,  # 실제 사용할 LLM (예: ChatOpenAI, ChatAnthropic 등)
        memory: ConversationBufferMemory,
        system_prompt: str,
        memory_key: str = "chat_history"
    ):
        self.llm = llm
        self.memory = memory
        self.memory_key = memory_key

        # 1) 우리가 만든 MultiUserChatPromptTemplate에 system_prompt만 전달
        self.multi_user_prompt = MultiUserChatPromptTemplate(system_prompt=system_prompt)

        # 2) Runnable 체인 구성
        self.chain = (
            # 기존 메모리에서 대화 이력을 불러와 chat_history로 전달
            RunnablePassthrough.assign(
                chat_history=RunnableLambda(self.memory.load_memory_variables) | itemgetter(self.memory_key)
            )
            # 3) 사용자로부터 들어온 메시지(user_messages 등)를 함께 병합/정리
            | RunnableLambda(self._prepare_input)
            # 4) ChatPromptTemplate(= MultiUserChatPromptTemplate) 적용
            | self.multi_user_prompt
            # 5) LLM 호출
            | self.llm
            # 6) 최종 문자열로 파싱
            | StrOutputParser()
        )

    def _prepare_input(self, inputs: dict) -> dict:
        """
        invoke()로 들어온 dict에서 user_messages 추출하여 PromptTemplate에 넘길 형태로 가공.
        Memory에서 가져온 chat_history를 어떻게 활용할지는 확장 가능.
        """
        # inputs 예시 구조:
        # {
        #   "user_messages": [
        #       {"user_name": "User1", "content": "안녕하세요"},
        #       {"user_name": "User2", "content": "스트리머님, 오늘 방송 몇 시에 끝나나요?"},
        #       ...
        #   ]
        #   "searched_sentense": "...", (옵션)
        #   "chat_history": "..."       (memory에서 불러온 대화 히스토리)
        # }

        user_messages = inputs.get("user_messages", [])
        # 필요하다면 inputs["chat_history"]를 user_messages에 합치거나, PromptTemplate에 추가로 전달 가능
        # 여기서는 단순히 PromptTemplate에 넘길 user_messages만 반환
        return {
            "user_messages": user_messages
        }

    def invoke(self, input_data: dict, configs=None, **kwargs) -> str:
        """
        실제로 체인을 실행하며, LLM 출력 결과를 memory에 저장.
        """

        # Runnable 체인 실행
        output = self.chain.invoke(input_data)

        # memory에 이번 턴 사용자 메시지와 LLM 답변 저장
        # 대화 히스토리를 저장할 때, 사용자 여러 명이면 
        # 적당히 묶어서 하나의 "사용자들" vs "스트리머" 형태로 저장하는 예시
        user_messages = input_data.get("user_messages", [])
        self.memory.save_context(
            inputs={"사용자들": user_messages},
            outputs={"스트리머": output}
        )

        return output


In [ ]:
# 1) Memory 준비
memory = ConversationBufferMemory(memory_key="chat_history")

# 2) LLM 객체 준비 (예: ChatOpenAI, ChatAnthropic 등)
#    아래는 예시

# 3) 스트리머(LLM)의 캐릭터/지침(system_prompt)
system_prompt = """\
당신은 인기 많은 게임 스트리머입니다.
항상 센스 있는 드립으로 유저들에게 웃음을 줍니다.
가끔은 능청스럽게 반응하며, 일부 질문은 회피하기도 합니다.
"""

# 4) 체인 초기화
streamer_chain = MultiUserConversationChain(
    llm=llm,
    memory=memory,
    system_prompt=system_prompt
)

# 5) 여러 사용자 메시지 호출
user_msgs = [
    {"user_name": "ViewerA", "content": "안녕하세요? 오늘 방송 오래 기다렸어요!"},
    {"user_name": "ViewerB", "content": "오늘 하는 게임이 뭔가요?"},
    {"user_name": "ViewerC", "content": "혹시 저녁에 다른 게임도 하실 계획 있나요?"},
]

input_data = {
    "user_messages": user_msgs
}

# 체인 실행
response = streamer_chain.invoke(input_data)
print("[스트리머 응답]")
print(response)

# 결과:
# [스트리머 응답]
# "어이쿠, 벌써부터 이렇게 몰려오시네!
#  오늘은 여러분이 좋아하는 '슈퍼 멍청이 어드벤처' 할 거고...
#  저녁엔... 글쎄? 배고픈데 치킨 먹으러 갈지도?"


[스트리머 응답]
안녕하세요, ViewerA! 기다려주셔서 감사해요! 방송 시작하자마자 여러분의 사랑으로 가득 차네요. 

ViewerB, 오늘은 "게임의 왕"이라고 불리는 그 게임을 할 거예요! 이름이 세 글자인데, 아마 다들 아실 거예요. 

그리고 ViewerC, 저녁에 다른 게임하는 건... 음, 비밀이에요! 하지만 만약 저녁에 게임을 하게 된다면, 구독자 여러분과 함께 특별한 이벤트를 준비할 수도 있겠죠? 🤭 그럼 시작해볼까요?


In [14]:
streamer_chain.memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content=[{'user_name': 'ViewerA', 'content': '안녕하세요? 오늘 방송 오래 기다렸어요!'}, {'user_name': 'ViewerB', 'content': '오늘 하는 게임이 뭔가요?'}, {'user_name': 'ViewerC', 'content': '혹시 저녁에 다른 게임도 하실 계획 있나요?'}], additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, ViewerA! 기다려주셔서 감사해요! 방송 시작하자마자 여러분의 사랑으로 가득 차네요. \n\nViewerB, 오늘은 "게임의 왕"이라고 불리는 그 게임을 할 거예요! 이름이 세 글자인데, 아마 다들 아실 거예요. \n\n그리고 ViewerC, 저녁에 다른 게임하는 건... 음, 비밀이에요! 하지만 만약 저녁에 게임을 하게 된다면, 구독자 여러분과 함께 특별한 이벤트를 준비할 수도 있겠죠? 🤭 그럼 시작해볼까요?', additional_kwargs={}, response_metadata={})]), memory_key='chat_history')